# Подготовка базы данных

In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect('window.db')

In [3]:
cursor = con.cursor()

In [4]:
sql = '''

create table employees (
    id integer primary key,
    name varchar(50),
    city varchar(50),
    department varchar(50),
    salary integer
);

insert into employees
(id, name, city, department, salary)
values
(24, 'Марина', 'Москва', 'it', 104),
(21, 'Елена', 'Самара', 'it', 84),
(22, 'Ксения', 'Москва', 'it', 90),
(25, 'Иван', 'Москва', 'it', 120),
(23, 'Леонид', 'Самара', 'it', 104),
(11, 'Дарья', 'Самара', 'hr', 70),
(12, 'Борис', 'Самара', 'hr', 78),
(31, 'Вероника', 'Москва', 'sales', 96),
(33, 'Анна', 'Москва', 'sales', 100),
(32, 'Григорий', 'Самара', 'sales', 96);

create table expenses (
    year integer,
    month integer,
    income integer,
    expense integer
);

insert into expenses
(year, month, income, expense)
values
(2020, 1, 94, 82),
(2020, 2, 94, 75),
(2020, 3, 94, 104),
(2020, 4, 100, 94),
(2020, 5, 100, 99),
(2020, 6, 100, 105),
(2020, 7, 100, 95),
(2020, 8, 100, 110),
(2020, 9, 104, 104);'''

In [5]:
cursor.executescript(sql)

OperationalError: table employees already exists

In [6]:
sql = '''select * from employees'''

pd.read_sql(sql, con)

,id,name,city,department,salary
0,11,Дарья,Самара,hr,70
1,12,Борис,Самара,hr,78
2,21,Елена,Самара,it,84
3,22,Ксения,Москва,it,90
4,23,Леонид,Самара,it,104
5,24,Марина,Москва,it,104
6,25,Иван,Москва,it,120
7,31,Вероника,Москва,sales,96
8,32,Григорий,Самара,sales,96
9,33,Анна,Москва,sales,100


In [7]:
# Напишем функцию, чтобы удобнее было возвращать
def select(sql):
    return pd.read_sql(sql, con)

In [8]:
select(sql)

,id,name,city,department,salary
0,11,Дарья,Самара,hr,70
1,12,Борис,Самара,hr,78
2,21,Елена,Самара,it,84
3,22,Ксения,Москва,it,90
4,23,Леонид,Самара,it,104
5,24,Марина,Москва,it,104
6,25,Иван,Москва,it,120
7,31,Вероника,Москва,sales,96
8,32,Григорий,Самара,sales,96
9,33,Анна,Москва,sales,100


# Задача 1

**Посчитать, сколько процентов составляет зарплата сотрудника от средней по Москве и средней по Самаре. :**

In [9]:
sql = '''
select t.id, t.name, t.salary,
round((t.salary)*100.0 / avg(t.salary) over(), 0) as perc,
round((t.salary * 100.0 / avg(t.salary) filter(where t.city = 'Москва') over()), 0) as msk,
round((t.salary * 100.0 / avg(t.salary) filter(where t.city = 'Самара') over()), 0) as sam
from employees as t
order by t.id
'''
select(sql)

,id,name,salary,perc,msk,sam
0,11,Дарья,70,74.0,69.0,81.0
1,12,Борис,78,83.0,76.0,90.0
2,21,Елена,84,89.0,82.0,97.0
3,22,Ксения,90,96.0,88.0,104.0
4,23,Леонид,104,110.0,102.0,120.0
5,24,Марина,104,110.0,102.0,120.0
6,25,Иван,120,127.0,118.0,139.0
7,31,Вероника,96,102.0,94.0,111.0
8,32,Григорий,96,102.0,94.0,111.0
9,33,Анна,100,106.0,98.0,116.0


# Задача 2

**Посчитать зарплатный фонд города без учета IT-отдела::**

In [10]:
sql = '''
select
  name, city,
  sum(salary) over w as base,
  sum(salary) filter(where department <> 'it') over w as no_it
from employees
window w as (partition by city)
order by city, id;

'''
select(sql)

,name,city,base,no_it
0,Ксения,Москва,510,196
1,Марина,Москва,510,196
2,Иван,Москва,510,196
3,Вероника,Москва,510,196
4,Анна,Москва,510,196
5,Дарья,Самара,432,244
6,Борис,Самара,432,244
7,Елена,Самара,432,244
8,Леонид,Самара,432,244
9,Григорий,Самара,432,244


# Задача 3

**Расчитать фонд зарплаты по городу. Добавьте столбец alt с зарплатным фондом, рассчитанным по следующим правилам:**

* для сотрудников отдела hr умножаем зарплату на 2;
* для сотрудников отдела it делим зарплату на 2;
* для сотрудников отдела sales не меняем зарплату.

In [11]:
sql = '''
select t.name, t.city,
sum(t.salary) over(partition by t.city) as base,
sum(case when t.department = 'hr' then t.salary*2.0
when t.department = 'it' then t.salary / 2.0
else t.salary end) over(partition by t.city) as alt
from employees as t


'''
select(sql)

,name,city,base,alt
0,Ксения,Москва,510,353.0
1,Марина,Москва,510,353.0
2,Иван,Москва,510,353.0
3,Вероника,Москва,510,353.0
4,Анна,Москва,510,353.0
5,Дарья,Самара,432,486.0
6,Борис,Самара,432,486.0
7,Елена,Самара,432,486.0
8,Леонид,Самара,432,486.0
9,Григорий,Самара,432,486.0
